# Who votes more with the right? (Summer Interview with Friedrich Merz)

Who votes with whom and which political groups are the building-blocks of majorities inside Parliament are often questions of special interest - not only for researchers, but especially when it comes to public discourse.

In his "Summer Interview" with the German public broadcaster ARD, Friedrich Merz said that the Social Democrats had voted in line with the groups on the right more often than the EPP group. In this tutorial, we will take a look at how to actually calculate this and conduct our very own fact-check.

To follow along with the tutorial, you should already be familiar with data analysis in Python using `pandas`. You don’t need prior knowledge about the European Parliament.

## Definitions

The trickiest part conducting an analysis such as this is defining what it means "to vote with someone". Often, similar analysis focuses on which parties form majorities together.
"Voting together" however does not necessarily imply that the shared vote is successful, but that the groups position on a vote are similar.

We propose to define the groups position to be the position that most members of a group aligned with. For example, the group position of a group where 10 members voted in favor, 8 against and 3 abstained would be in favor.
Voting together with another group then means sharing the same group position on any given vote.

Note that this is only one of many possible approaches. We decided on this approach, as Merz's statement did not mention majorities or a successful text, but instead referred to groups voting together, which we feel is best captured by condensing all votes into a concrete group position.

Merz's statement referred to _groups on the right_. We consider that to be the ECR, PfE, and ESN groups, as these are the groups considered to be on the right of the EPP. We will focus our analysis on the 10th term of Parliament, as this is the term during which Merz made his claim. 

In the following, we will now find out how often S&D and EPP shared a position with these right-wing groups.

### General Remarks on Votes in the European Parliament

It is important to keep in mind that not all votes in the European Parliament are recorded in a way that allows to check how each individual MEP voted.
Votes where this is possible are called roll-call votes. They are quite common in the European Parliament, but not ubiquitous. Naturally, for votes who are not taken by roll-call it is impossible for us (or anyone else) to know exactly which MEPs and groups voted similar.

## The Plan

We will follow these steps:

1. Find all votes which took place during the 10th term of Parliament and exclude some special cases like votes on the agenda.
2. For each vote, construct a table in which we sum up the votes per group per position. This will result in a table where we have one row per vote-group-position combination, and the number of MEPs who voted that way.
3. Using this table, we can find which position per vote was most common for members of each group - this is what we will call the group-line or group position.
4. The tables up until now were in long format. We will construct a table in wide-format next, with a single row per vote and a column for each party, containing the group-line.
5. Finally, we can compare the columns, allowing us to count how often the group-line between different groups was similar.

## Filtering for Relevant Votes

In [ ]:
import pandas as pd
# The votes table contains one row per roll-call-vote taken in the European Parliament
votes_df = pd.read_csv('data/votes.csv')
votes_df.head()

In [ ]:
votes_df['timestamp'] = pd.to_datetime(votes_df['timestamp'], format='ISO8601')
# Filter votes so that only votes during the 10th term and before Merz's statement are kept
mask_time = (votes_df["timestamp"] >= "2024-07-16") & (votes_df["timestamp"] <= "2025-07-13")
votes_subset = votes_df.loc[mask_time]

# Filter votes to exclude special cases like votes on the agenda
votes_subset  = votes_subset[~votes_subset['display_title'].str.contains('Ordre du jour|Demande du group|agenda')]
votes_subset.head()

The `votes` table does not contain information about the individual votes of MEPs. This can be found in the `member_votes` table. This table has long-format, so a single row per vote per MEP, containing their vote, group-membership at the time of the vote, and their nationality.

In [ ]:
member_votes = pd.read_csv('data/member_votes.csv')
member_votes.head()

We use a filtering-join with the `votes_subset` to only keep votings on the votes relevant to our analysis:

In [ ]:
member_votes_subset = member_votes[member_votes["vote_id"].isin(votes_subset["id"])]
member_votes_subset.head()

Before we start aggregating the MEPs votes per group and position to find out the group-line per vote, we can exclude a lot of votings from our `member_votes_subset`: As we are only interested in the voting behavior of the EPP and S&D groups, as well as the three right-wing group (ECR, PfE, ESN), we can exclude all votes made by members of other groups:

In [ ]:
member_votes_subset = member_votes_subset[member_votes_subset["group_code"].isin(["EPP", "SD", "ECR", "PFE", "ESN"])]
member_votes_subset.head()

As a next step, we build a table with one row per vote-group-position combination and the number of MEPs corresponding to this combination.
Technically, this comes down to grouping the data by vote_id, group_code and position, and then adding the size of the group as a new column.

In [ ]:
group_counts = member_votes_subset \
                .groupby(["vote_id", "group_code", "position"]) \
                .size() \
                .reset_index(name="count")

group_counts

Now we know which number of MEPs per group took which position on each vote, we can identify which position **most** MEPs of a group took.
The code below achieves this by first sorting by number of MEPs in ascending order (so that the highest number of MEPs per vote-group-position combination comes first) and afterwards removing duplicates on the basis of vote-group combinations, so that only the very first entry per vote-group combination stays, which will be the one with the highest number of MEPs.

In [ ]:
group_position = group_counts \
    .sort_values("count", ascending=False) \
    .drop_duplicates(subset=["vote_id","group_code"])

group_position

As a last step, we will pivot this table into wide format, so that we have a single row per vote, and the group-position of each group in the columns. This makes it easy to compare the positions on each vote by group.

In [ ]:
group_position_comparison = (
    group_position
    .pivot(index="vote_id", columns="group_code", values="position")
    .reset_index()
)
group_position_comparison

When can now easily compare two groups with each other. We will construct one column each for the S&D and EPP group, that contains True when their respective group-position was the same as a specific right-wing group. To find out whether S&D or EPP more often aligned with any given right-wing group, we simply need to sum up this column and can compare the results.

We begin by comparing S&D and EPP with the ESN group:

In [ ]:
group_position_comparison["EPP_equals_ESN"] = group_position_comparison["EPP"] == group_position_comparison["ESN"]
group_position_comparison["SD_equals_ESN"]  = group_position_comparison["SD"]  == group_position_comparison["ESN"]

In [ ]:
sum(group_position_comparison["EPP_equals_ESN"])

In [ ]:
sum(group_position_comparison["SD_equals_ESN"])

In the 10th term of the European Parliament, the EPP voted similarly to the ESN more often than the S&D group

Next, we compare both to the PFE:

In [ ]:

group_position_comparison["SD_equals_PFE"]  = group_position_comparison["SD"]  == group_position_comparison["PFE"]
group_position_comparison["EPP_equals_PFE"] = group_position_comparison["EPP"] == group_position_comparison["PFE"]

In [ ]:
sum(group_position_comparison["EPP_equals_PFE"])

In [ ]:
sum(group_position_comparison["SD_equals_PFE"])

Again, the EPP voted together with PfE more often. Lastly, we compare to the ECR:

In [ ]:
group_position_comparison["EPP_equals_ECR"] = group_position_comparison["EPP"] == group_position_comparison["ECR"]
group_position_comparison["SD_equals_ECR"]  = group_position_comparison["SD"]  == group_position_comparison["ECR"]

In [ ]:
sum(group_position_comparison["EPP_equals_ECR"])

In [ ]:
sum(group_position_comparison["SD_equals_ECR"])

The same is true for the ECR - the group-line of the EPP was more often aligned with the group-line of the ECR than that of the S&D group.


## Conclusion

Considering only roll-call votes of the 10th term of the European Parliament, the EPP voted together with right-wing groups more often than the S&D group. To the extent Merz's claim is verifiable, it is false.